# SCRAPING & GEOLOCATING

In [10]:
from selenium import webdriver
import requests
from requests import get
from bs4 import BeautifulSoup
from IPython.core.display import clear_output
import pandas as pd
from random import randint
from time import sleep

In [3]:
URL='https://www.astikaakinita.gr/anazitisi-akiniton/'

ΔΟΚΙΜΑΣΤΙΚΑ ΣΚΡΑΠΑΡΟΥΜΕ ΤΗΝ ΠΡΩΡΤΗ ΣΕΛΙΔΑ

In [5]:
response=get(URL)
doc=BeautifulSoup(response.text, "html.parser")
conteiner=doc.find('div', attrs={'class':'realastate_search_results flex-row'})
contents=[]
for item in conteiner.findAll('article',attrs={'class':'realastate_item'}):
    
    estate={
        'type_holding':item.h4.text,
        'type':item.h3.text,
        'addr':item.p.text,
        'code':item('em',attrs={'class':'ref_code'})[0].text,
        'sq':item('span',attrs={'class':'auction_text'})[0].text,
        'link':item.a.get('href')
        
    }
    price=item('p',attrs={'class':'price'})
    
#     Εάν υπάρχει τιμή, τότε να την σώσουμε στο dict
    if len(price)>0:
        estate={'price':price[0].em.text}
    contents.append(estate)

In [ ]:
# Εμφανίζουμε τι πήραμε
# contents

### ΤΡΑΒΑΜΕ ΟΛΕΣ ΤΙΣ ΣΕΛΙΔΕΣ ΤΩΝ ΑΣΤΙΚΩΝ ΑΚΙΝΗΤΩΝ

In [17]:
PAGE_LINKS=['https://www.astikaakinita.gr/anazitisi-akiniton/?pg='+str(item) for item in range(1,42)]

In [18]:
contents=[]
page_no=0
estate_no=0
for page in PAGE_LINKS:
    page_no+=1
    response=get(page)
    doc=BeautifulSoup(response.text, "html.parser")
    conteiner=doc.find('div', attrs={'class':'realastate_search_results flex-row'})
    
    for item in conteiner.findAll('article',attrs={'class':'realastate_item'}):
        estate_no+=1
        estate={
        'type_holding':item.h4.text,
        'type':item.h3.text,
        'addr':item.p.text,
        'code':item('em',attrs={'class':'ref_code'})[0].text,
        'link':item.a.get('href')
        }
                
        sq=item('span',attrs={'class':'auction_text'})
        if len(sq)>0:
            estate['sq']=sq[0].text
        else:
             estate['sq']='UNK'
                
        price=item('p',attrs={'class':'price'})
        if len(price)>0:
            estate['price']=price[0].em.text
        
        contents.append(estate)
        clear_output(wait=True)
        print('estate '+ str(estate_no))
#         sleep(randint(1,2))
    clear_output(wait=True)
    print('page '+ str(page_no))

page 41


In [20]:
# contents

ΕΝΤΟΠΙΖΟΥΜΕ ΤΙΣ ΣΥΝΤΕΤΑΓΜΕΝΕΣ ΕΝΟΣ ΑΚΙΝΗΤΟΥ

In [21]:
api_key=''

In [22]:
geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(contents[1]['addr'])+'&key='+api_key
results = requests.get(geocode_url)
res=results.json()
if res['status']!='ZERO_RESULTS':
    lat=res['results'][0]['geometry']['location']['lat']
    lng=res['results'][0]['geometry']['location']['lng']
    print(lat)
    print(lng)


37.8833564
23.9333037


ΕΝΤΟΠΙΖΟΥΜΕ ΤΙΣ ΣΥΝΤΕΤΑΓΜΕΝΕΣ ΤΩΝ ΥΠΟΛΟΙΠΩΝ

In [23]:
counter = 0
for content in contents:
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(content['addr'])+'&key='+api_key
    results = requests.get(geocode_url)
    res=results.json()
    counter +=1
    if res['status']!='ZERO_RESULTS':
        lat=res['results'][0]['geometry']['location']['lat']
        lng=res['results'][0]['geometry']['location']['lng']
        content['lat']=lat
        content['lng']=lng
    else:
        content['lat']='UNK'
        content['lng']='UNK'
    print("Geolocating realestate {} of {}".format(counter, len(contents)))
    clear_output(wait=True)

Geolocating realestate 478 of 478


ΑΠΟΘΗΚΕΥΟΥΜΕ ΤΗ ΒΑΣΗ ΔΕΔΟΜΕΝΩΝ

In [41]:
pd.DataFrame(contents)\
.replace('\r\n','',regex=True)\
.replace('€','',regex=True)\
.replace('τ.μ.','',regex=True)\
.to_csv('geolocated_realestate.csv',index=False)